# Imports

In [1]:
from pathlib2 import Path
from riotwatcher import LolWatcher, ApiError
import json

# Functions
## API Key

In [2]:
def api_key(credentials):
    """Checking if the development API key from riot's development portal has been set and/or needs to be updated
    
    Input is a JSON file 

    """
    with open(credentials) as json_data:
        credentials = json.load(json_data)
        api_key = credentials["dev_api_key"]
        lol_watcher = LolWatcher(credentials["dev_api_key"])
        try:
            server_status = lol_watcher.lol_status.shard_data("euw1")
        except ApiError as error:
            if error.response.status_code == 403:
                print("the API key is no good bro")
            # elif error.response.status_code == 404: 
            #     print("Found no match")
            else:
                return credentials["dev_api_key"]
   
    # TODO: add that it checks for if the key is still valid or not error 404 or something and if so enter it and it updates the dev_api_key.json file
    # global api_key
    #     api_key # run something like check if server is online https://developer.riotgames.com/apis#lol-status-v4/GET_getPlatformData
    # except NameError:
    #     api_key = str(input("Input development API key."))
    # else:
    #     print("API key already defined.")
    

In [23]:
api_key_loc = data_folder / "dev_api_key.json"
with open(api_key_loc) as json_data:
    credentials = json.load(json_data)
    api_key = credentials["dev_api_key"]
    lol_watcher = LolWatcher(credentials["dev_api_key"])
    try:
        server_status = lol_watcher.lol_status.shard_data("euw1")
        print("it works") # return credentials["dev_api_key"]
    except ApiError as error:
        if error.response.status_code == 403:
            print("the API key is no good bro")
            new_api_key = input("Enter correct API key here.")
            credentials["dev_api_key"] = new_api_key 
            print(new_api_key)

the API key is no good bro
RGAPI-2e60d09f-0ce6-4485-ab5b-223c3352b6f6


In [28]:
# Test API key
api_key_loc = data_folder / "dev_api_key.json"
# lol_watcher = LolWatcher(api_key(api_key_loc)) #the test

lol_watcher = LolWatcher("RGAPI-2e60d09f-0ce6-4485-ab5b-223c3352b6f6") # check
if lol_watcher.lol_status.shard_data("euw1"):
    print("it worked")

it worked


# Folder locations

In [29]:
# Folder locations

# Project folders
project_folder = Path.cwd().parent

# Raw data storage
data_storagez= project_folder / "data"

# Out folder
out_folder = project_folder / "out"

In [30]:
# Set API key
api_key_loc = project_folder / "dev_api_key.json"
lol_watcher = LolWatcher(api_key(api_key_loc)) 

TypeError: 'str' object is not callable

# Calculating KDA


In [1]:
Defining variables

KDA_Match: KDA calculated per match per player
KDA_Season: Mean KDA for that player for this seasons

K_Match: Total kills in that match
D_Match: Total deaths in that match
A_Match: Total assists in that match

K_Season: Total kills in that season
D_Season: Total deaths in that season
A_Season: Total assists in that season
? Should we use total or average?

Calculations

KDA base calculation= Kills + Assists / Deaths
KDA_Match = K_Match + A_Match / D_Match
KDA_Season = K_Season + A_Season / D_Season


Setting things up: 
Season: sets the season we are pulling data from
Region: sets the region we are pulling data from



SyntaxError: invalid syntax (<ipython-input-1-263b48d9eeac>, line 2)

In [ ]:
# set match request parameters

# Set queue
queue = 420

# Set season

In [136]:
# Retrieve last 100 matches (max is 100 matches https://developer.riotgames.com/apis#match-v4) but time range is 1 week 
matches = lol_watcher.match.matchlist_by_account(region=region,
                                                 encrypted_account_id=user["accountId"],
                                                 queue=queue,
                                                 begin_index=0,
                                                 end_index=100)
# print(matches.keys())

# Get match details
for i, match in enumerate(matches["matches"][0:1]): # currently last 5 matches
        print(i, match)
        match_detail = lol_watcher.match.by_id(region, match['gameId'])
        
participants = []
for row in match_detail['participants']:
    participants_row = {}
    participants_row['champion'] = row['championId']
    participants_row['spell1'] = row['spell1Id']
    participants_row['spell2'] = row['spell2Id']
    participants_row['win'] = row['stats']['win']
    participants_row['kills'] = row['stats']['kills']
    participants_row['deaths'] = row['stats']['deaths']
    participants_row['assists'] = row['stats']['assists']
    participants_row['KDA'] = (row['stats']['kills']+row['stats']['assists'])/row['stats']['deaths']
    participants_row['totalDamageDealt'] = row['stats']['totalDamageDealt']
    participants_row['goldEarned'] = row['stats']['goldEarned']
    participants_row['champLevel'] = row['stats']['champLevel']
    participants_row['totalMinionsKilled'] = row['stats']['totalMinionsKilled']
    participants_row['item0'] = row['stats']['item0']
    participants_row['item1'] = row['stats']['item1']
    participants.append(participants_row)
df = pd.DataFrame(participants)
df

0 {'platformId': 'EUW1', 'gameId': 4994448524, 'champion': 202, 'queue': 420, 'season': 13, 'timestamp': 1608750682906, 'role': 'SOLO', 'lane': 'BOTTOM'}


,champion,spell1,spell2,win,kills,deaths,assists,KDA,totalDamageDealt,goldEarned,champLevel,totalMinionsKilled,item0,item1
0,245,14,4,False,3,12,2,0.416667,86328,6624,12,117,2033,3152
1,202,4,7,False,2,7,6,1.142857,52629,7906,11,113,1055,3009
2,75,4,12,False,6,4,6,3.000000,82125,10014,14,130,1054,3110
3,28,4,11,False,9,9,3,1.333333,117825,10215,13,20,1082,3100
4,99,4,14,False,2,5,4,1.200000,42003,6743,11,63,3860,4005
5,101,4,14,True,10,5,3,2.600000,48057,9959,12,40,3853,3020
6,777,4,14,True,12,5,8,4.000000,165417,12944,16,175,1055,6673
7,33,4,11,True,5,4,16,5.250000,97872,9997,14,14,3068,3111
8,67,4,12,True,5,6,10,2.500000,87816,10465,14,134,1083,3009
9,145,4,7,True,5,2,8,6.500000,76364,10332,13,159,1055,6671


# Some extra info

In [ ]:
# How a request link looks like and what all the parts do

# https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Doublelift?api_key=RGAPI-edd56035-f7c2-4e06-83d4-3b578380b730

# https://- All requests to the API must use HTTPS. This ensures maximum security on your requests.
# na1– This is the “Platform” that you’re making the request on. For example, NA summoners will use NA1, EUW summoners will use EUW1. See the Regional Endpoints page for more platform IDs. For now, just understand that a platform is related to the region that the summoner plays on
# api.riotgames.com – This means we’re accessing the Riot Games API
# lol/summoner/v3/summoners/by-name/– The API we're wanting data from, in this case summoners by name
# WxWatch – the parameter for the endpoint, in this case, the name of the summoner we want data for (it's me!)
# ?api_key – Our API key which tells the API that we are allowed to get data
